In [1]:
import pandas as pd
import numpy as np

In [2]:
# Crear datos de ejemplo con problemas
ventas = pd.DataFrame({
    'producto': ['A', 'B', None, 'A', 'C'],
    'precio': [100, None, 150, 100, 200],
    'cantidad': [1, 2, None, 1, 3],
    'fecha': ['2024-01-01', None, '2024-01-03', '2024-01-01', 'invalid']
})

print("Datos originales:")
print(ventas)

print("\nValores faltantes por columna:")
print(ventas.isnull().sum())

Datos originales:
  producto  precio  cantidad       fecha
0        A   100.0       1.0  2024-01-01
1        B     NaN       2.0        None
2     None   150.0       NaN  2024-01-03
3        A   100.0       1.0  2024-01-01
4        C   200.0       3.0     invalid

Valores faltantes por columna:
producto    1
precio      1
cantidad    1
fecha       1
dtype: int64


In [3]:
def limpiar_datos_ventas(df):
    df_limpio = df.copy()
    
    # 1. Eliminar duplicados
    df_limpio = df_limpio.drop_duplicates()
    
    # 2. Imputar valores faltantes
    df_limpio['precio'] = df_limpio['precio'].fillna(df_limpio['precio'].median())
    df_limpio['cantidad'] = df_limpio['cantidad'].fillna(1)  # Asumir cantidad mínima
    
    # 3. Eliminar filas con producto faltante
    df_limpio = df_limpio.dropna(subset=['producto'])
    
    # 4. Corregir fechas inválidas
    df_limpio['fecha'] = pd.to_datetime(df_limpio['fecha'], errors='coerce')
    df_limpio = df_limpio.dropna(subset=['fecha'])
    
    # 5. Calcular total de la venta
    df_limpio['total'] = df_limpio['precio'] * df_limpio['cantidad']
    
    return df_limpio

In [4]:
ventas_limpias = limpiar_datos_ventas(ventas)

print("Datos limpios:")
print(ventas_limpias)

print(f"\nRegistros finales: {len(ventas_limpias)}")

Datos limpios:
  producto  precio  cantidad      fecha  total
0        A   100.0       1.0 2024-01-01  100.0

Registros finales: 1


In [6]:
def validar_ventas_limpias(df):
    validaciones = {
        'sin_faltantes': df.isnull().sum().sum() == 0,
        'precios_positivos': (df['precio'] > 0).all(),
        'cantidades_positivas': (df['cantidad'] > 0).all(),
        'fechas_validas': pd.api.types.is_datetime64_any_dtype(df['fecha']),
        'total_correcto': np.allclose(df['total'], df['precio'] * df['cantidad'])
    }
    
    print("Validaciones:")
    for check, passed in validaciones.items():
        status = "✅" if passed else "❌"
        print(f"  {status} {check}")
    
    return all(validaciones.values())

In [7]:
es_valido = validar_ventas_limpias(ventas_limpias)
print(f"\nDataset válido: {es_valido}")

Validaciones:
  ✅ sin_faltantes
  ✅ precios_positivos
  ✅ cantidades_positivas
  ✅ fechas_validas
  ✅ total_correcto

Dataset válido: True


In [8]:
import pandas as pd
import numpy as np

# =========================
# 1. Datos originales
# =========================
ventas = pd.DataFrame({
    'producto': ['A', 'B', None, 'A', 'C'],
    'precio': [100, None, 150, 100, 200],
    'cantidad': [1, 2, None, 1, 3],
    'fecha': ['2024-01-01', None, '2024-01-03', '2024-01-01', 'invalid']
})

valores_faltantes = ventas.isnull().sum().reset_index()
valores_faltantes.columns = ['columna', 'valores_faltantes']

# =========================
# 2. Limpieza de datos
# =========================
def limpiar_datos_ventas(df):
    df_limpio = df.copy()

    # Eliminar duplicados
    df_limpio = df_limpio.drop_duplicates()

    # Imputar valores faltantes
    df_limpio['precio'] = df_limpio['precio'].fillna(df_limpio['precio'].median())
    df_limpio['cantidad'] = df_limpio['cantidad'].fillna(1)

    # Eliminar producto faltante
    df_limpio = df_limpio.dropna(subset=['producto'])

    # Corregir fechas
    df_limpio['fecha'] = pd.to_datetime(df_limpio['fecha'], errors='coerce')
    df_limpio = df_limpio.dropna(subset=['fecha'])

    # Calcular total
    df_limpio['total'] = df_limpio['precio'] * df_limpio['cantidad']

    return df_limpio

ventas_limpias = limpiar_datos_ventas(ventas)

# =========================
# 3. Validaciones
# =========================
validaciones = {
    'sin_faltantes': ventas_limpias.isnull().sum().sum() == 0,
    'precios_positivos': (ventas_limpias['precio'] > 0).all(),
    'cantidades_positivas': (ventas_limpias['cantidad'] > 0).all(),
    'fechas_validas': pd.api.types.is_datetime64_any_dtype(ventas_limpias['fecha']),
    'total_correcto': np.allclose(
        ventas_limpias['total'],
        ventas_limpias['precio'] * ventas_limpias['cantidad']
    )
}

df_validaciones = pd.DataFrame(
    list(validaciones.items()),
    columns=['validacion', 'resultado']
)

# =========================
# 4. Exportar a Excel
# =========================
ruta_excel = "evidencia_limpieza_validacion_dia2.xlsx"

with pd.ExcelWriter(ruta_excel) as writer:
    ventas.to_excel(writer, sheet_name='datos_originales', index=False)
    valores_faltantes.to_excel(writer, sheet_name='valores_faltantes', index=False)
    ventas_limpias.to_excel(writer, sheet_name='datos_limpios', index=False)
    df_validaciones.to_excel(writer, sheet_name='validaciones', index=False)

print("Archivo Excel generado correctamente:")
print(ruta_excel)

Archivo Excel generado correctamente:
evidencia_limpieza_validacion_dia2.xlsx
